# Coco Detection

## Sotabench Example

In [1]:
import sotabench
import torchvision

In [2]:
model = torchvision.models.detection.__dict__['maskrcnn_resnet50_fpn'](num_classes=91, pretrained=True)
model.to('cuda')

MaskRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d()
          )
        )
    

In [3]:
import pycocotools

In [4]:
from sotabench.object_detection.transforms import Compose, ConvertCocoPolysToMask, ToTensor
from sotabench.object_detection import COCO

test_results = COCO.benchmark(data_root='./.data/vision', batch_size=8, model=model)

loading annotations into memory...
Done (t=0.55s)
creating index...
index created!
625
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255

## Sotabench internal testing

In [4]:
from sotabench.object_detection.transforms import Compose, ConvertCocoPolysToMask, ToTensor

transforms = Compose([ConvertCocoPolysToMask(), ToTensor()])


class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, root, annFile, transforms):
        super(CocoDetection, self).__init__(root, annFile)
        self._transforms = transforms

    def __getitem__(self, idx):
        img, target = super(CocoDetection, self).__getitem__(idx)
        image_id = self.ids[idx]
        target = dict(image_id=image_id, annotations=target)
        if self._transforms is not None:
            img, target = self._transforms(img, target)
        return img, target


In [5]:
import os
from torch.utils.data import DataLoader

from sotabench.core import BenchmarkResult
from sotabench.utils import send_model_to_device

from sotabench.object_detection.utils import collate_fn, evaluate_detection_coco

device: str = 'cuda'
data_root: str = './.data/vision'
num_workers: int = 4
batch_size: int = 1
num_gpu: int = 1
paper_model_name: str = None
paper_arxiv_id: str = None
paper_pwc_id: str = None
pytorch_hub_url: str = None

model, device = send_model_to_device(model, device=device, num_gpu=num_gpu)
model.eval()

test_dataset = CocoDetection(root=os.path.join(data_root, 'val2017'),
                                   annFile=os.path.join(data_root, 'annotations/instances_val2017.json'),
                                      transforms=transforms)

loading annotations into memory...
Done (t=0.56s)
creating index...
index created!


In [6]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True,
                         collate_fn=collate_fn)
test_loader.no_classes = 91  # Number of classes for COCO Detection

In [7]:
test_loader.no_classes = 91  # Number of classes for COCO Detection

In [9]:
import torch
import torchvision

from pycocotools import mask as coco_mask
from pycocotools.coco import COCO

from sotabench.object_detection.coco_eval import CocoEvaluator
from sotabench.object_detection.utils import get_coco_api_from_dataset, _get_iou_types


In [36]:
coco_evaluator.coco_eval['bbox'].summarize()

coco = get_coco_api_from_dataset(test_loader.dataset)
iou_types = _get_iou_types(model)
coco_evaluator = CocoEvaluator(coco, iou_types)
model_output_transform = None

with torch.no_grad():
    print(len(test_loader))
    for i, (input, target) in enumerate(test_loader):
        input = list(inp.to(device=device, non_blocking=True) for inp in input)
        target = [{k: v.to(device=device, non_blocking=True) for k, v in t.items()} for t in target]

        # compute output
        output = model(input)

        if model_output_transform is not None:
            output = model_output_transform(output, target)
        elif test_loader.no_classes == 91: # COCO
            output = [{k: v.to('cpu') for k, v in t.items()} for t in output]  # default torchvision extraction

        result = {tar["image_id"].item(): out for tar, out in zip(target, output)}
        coco_evaluator.update(result)
        if i > 200:
            break
            
coco_evaluator.synchronize_between_processes()
coco_evaluator.accumulate()
coco_evaluator.coco_eval['bbox'].summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.445
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.680
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.481
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.294
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.472
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.618
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.379
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.548
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.556
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.351
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.572
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.717
5000
Accumulating evaluation results...


In [71]:
import numpy as np

ap = 1
metrics = {'AP (BB)': None, 
           'AP (BB) 50': None, 'AP (BB) 75': None, 'AP (BB) S': None, 'AP (BB) M': None, 'AP (BB) L': None}
iouThrs = [None, .5, .75, None, None, None]
maxDets = [100] + [coco_evaluator.coco_eval['bbox'].params.maxDets[2]]*5
areaRngs = ['all', 'all', 'all', 'small', 'medium', 'large']
bounding_box_params = coco_evaluator.coco_eval['bbox'].params

for metric_no, metric in enumerate(metrics):
    aind = [i for i, aRng in enumerate(bounding_box_params.areaRngLbl) if aRng == areaRngs[metric_no]]
    mind = [i for i, mDet in enumerate(bounding_box_params.maxDets) if mDet == maxDets[metric_no]]

    s = coco_evaluator.coco_eval['bbox'].eval['precision']
    
    # IoU
    if iouThrs[metric_no] is not None:
        t = np.where(iouThrs[metric_no] == bounding_box_params.iouThrs)[0]
        s = s[t]
    s = s[:,:,:,aind,mind]
    
    if len(s[s>-1])==0:
        mean_s = -1
    else:
        mean_s = np.mean(s[s>-1])
        
    print(metric)
    metrics[metric] = mean_s

print(metrics)

AP (BB)
AP (BB) 50
AP (BB) 75
AP (BB) S
AP (BB) M
AP (BB) L
{'AP (BB)': 0.41933158241416846, 'AP (BB) 50': 0.6353941321524154, 'AP (BB) 75': 0.46899880017449536, 'AP (BB) S': 0.2845841673759698, 'AP (BB) M': 0.46122455937409707, 'AP (BB) L': 0.5836261989330561}


In [16]:
coco_evaluator.iou_types

['bbox', 'segm']

In [37]:
coco_evaluator.coco_eval['bbox'].params

In [20]:
def _summarize( ap=1, iouThr=None, areaRng='all', maxDets=100 ):
    p = coco_evaluator.params
    iStr = ' {:<18} {} @[ IoU={:<9} | area={:>6s} | maxDets={:>3d} ] = {:0.3f}'
    titleStr = 'Average Precision' if ap == 1 else 'Average Recall'
    typeStr = '(AP)' if ap==1 else '(AR)'
    iouStr = '{:0.2f}:{:0.2f}'.format(p.iouThrs[0], p.iouThrs[-1]) \
        if iouThr is None else '{:0.2f}'.format(iouThr)

    aind = [i for i, aRng in enumerate(p.areaRngLbl) if aRng == areaRng]
    mind = [i for i, mDet in enumerate(p.maxDets) if mDet == maxDets]
    if ap == 1:
        # dimension of precision: [TxRxKxAxM]
        s = coco_evaluator.eval['precision']
        # IoU
        if iouThr is not None:
            t = np.where(iouThr == p.iouThrs)[0]
            s = s[t]
        s = s[:,:,:,aind,mind]
    else:
        # dimension of recall: [TxKxAxM]
        s = coco_evaluator.eval['recall']
        if iouThr is not None:
            t = np.where(iouThr == p.iouThrs)[0]
            s = s[t]
        s = s[:,:,aind,mind]
    if len(s[s>-1])==0:
        mean_s = -1
    else:
        mean_s = np.mean(s[s>-1])
    print(iStr.format(titleStr, typeStr, iouStr, areaRng, maxDets, mean_s))
    return mean_s

In [21]:
_summarize(1)

AttributeError: 'CocoEvaluator' object has no attribute 'params'